REV-1.0.0 - LIB P\ OPERAR: BUSCAR ARQUIVO wpp.txt

In [3]:
import os
from google.colab import files

# Definir o caminho de destino
destino = "/content/arquivos-gerados/arquivos-txt/listas-txt"
os.makedirs(destino, exist_ok=True)  # Criar o diret√≥rio, se n√£o existir

# Fun√ß√£o para salvar o arquivo
def salvar_arquivo_txt():
    while True:  # Loop para garantir que o usu√°rio forne√ßa um arquivo v√°lido
        print("Aguarde a janela de sele√ß√£o abrir e escolha um arquivo .txt para continuar.")
        try:
            uploaded = files.upload()  # Abrir a janela de sele√ß√£o de arquivo
        except KeyboardInterrupt:
            # Caso o bot√£o 'Cancel upload' seja clicado
            print("Nenhum arquivo foi selecionado. Para tratar os dados, insira um arquivo .txt.")
            return

        if not uploaded:  # Caso nenhum arquivo tenha sido enviado (Cancel upload)
            print("Nenhum arquivo foi selecionado. Para tratar os dados, insira um arquivo .txt.")
            return

        for nome_arquivo, conteudo_arquivo in uploaded.items():
            # Verificar se o arquivo tem extens√£o .txt
            if not nome_arquivo.endswith(".txt"):
                print(f"Erro: O arquivo '{nome_arquivo}' n√£o √© um .txt.")
                print("Por favor, carregue um arquivo com extens√£o .txt para continuar.")

                # Excluir arquivos desnecess√°rios
                caminho_indesejado = os.path.join("/content", nome_arquivo)
                if os.path.exists(caminho_indesejado):
                    os.remove(caminho_indesejado)
                    print(f"Arquivo inv√°lido '{nome_arquivo}' exclu√≠do de /content/.")
                continue  # Volta para o in√≠cio do loop para pedir novamente

            # Caminho completo para salvar o arquivo renomeado como 'wpp.txt'
            caminho_destino = os.path.join(destino, "wpp.txt")

            # Salvar o arquivo no destino com o novo nome
            with open(caminho_destino, "wb") as novo_arquivo:
                novo_arquivo.write(conteudo_arquivo)

            print(f"Arquivo '{nome_arquivo}' salvo como 'wpp.txt' em {destino}")

            # Remover o arquivo original salvo em /content/
            caminho_original = os.path.join("/content", nome_arquivo)
            if os.path.exists(caminho_original):
                os.remove(caminho_original)
                print(f"Arquivo original '{nome_arquivo}' removido de /content/.")

            return  # Sai da fun√ß√£o ao processar o arquivo v√°lido

# Executar a fun√ß√£o diretamente
salvar_arquivo_txt()


ModuleNotFoundError: No module named 'google.colab'

**REV-2.0.0 - LIB. P\ OPERAR: FILTRAR RELAT√ìRIOS DO ARQUIVO WPP.TXT EM ARQUIVO RELATORIO_FILTRADO.txt**

In [4]:
import re
from pathlib import Path
from datetime import datetime

def carregar_estrutura_rdo(caminho_estrutura):
    """Carrega a estrutura do relat√≥rio a partir do arquivo estrutura-rdo.txt."""
    try:
        parametros = []
        with open(caminho_estrutura, "r", encoding="utf-8") as f:
            for linha in f:
                linha = linha.strip()
                if linha and not linha.startswith("-----"):  # Ignora separadores
                    parametros.append(linha.rstrip(":"))  # Remove ':' no final
        print(f"Estrutura carregada com {len(parametros)} par√¢metros.")
        return parametros
    except FileNotFoundError:
        print(f"Erro: Arquivo de estrutura '{caminho_estrutura}' n√£o encontrado.")
        return []

def filtrar_relatorio_completo(arquivo_entrada, arquivo_saida, parametros_estrutura, arquivo_irrelevante, minimo_parametros=10):
    """Filtra relat√≥rios completos com base na estrutura do arquivo estrutura-rdo.txt."""
    relatorio_em_construcao = []
    relatorios_filtrados = []
    parametros_usados = set()
    relatorio_valido = False
    linhas_irrelevantes = []  # Armazena as linhas irrelevantes

    try:
        with open(arquivo_entrada, "r", encoding="utf-8") as entrada:
            for linha in entrada:
                linha = linha.strip()
                print(f"Processando linha: {linha}")  # Log para depura√ß√£o

                # Identifica o in√≠cio de um relat√≥rio
                if linha.startswith("RETORNO DAS ATIVIDADES"):
                    # Adiciona o relat√≥rio anterior se atender ao m√≠nimo de par√¢metros
                    if relatorio_valido and len(parametros_usados) >= minimo_parametros:
                        relatorios_filtrados.append("\n".join(relatorio_em_construcao))
                        print(f"Relat√≥rio completo detectado: {relatorios_filtrados[-1]}")
                    elif relatorio_valido:
                        print("Aviso: Relat√≥rio incompleto ignorado.")

                    # Inicia um novo relat√≥rio
                    relatorio_em_construcao = [linha]
                    parametros_usados = {"RETORNO DAS ATIVIDADES"}
                    relatorio_valido = True
                    continue

                # Verifica se a linha cont√©m um par√¢metro v√°lido
                parametro_valido = False
                for parametro in parametros_estrutura:
                    if linha.lower().startswith(parametro.lower() + ":"):
                        relatorio_em_construcao.append(linha)
                        parametros_usados.add(parametro)
                        print(f"Par√¢metro detectado: {parametro}")  # Log para depura√ß√£o
                        parametro_valido = True
                        break

                # Se a linha n√£o for relevante, armazene como irrelevante
                if not parametro_valido and linha:
                    linhas_irrelevantes.append(linha)

                # Finaliza o relat√≥rio se a linha estiver vazia
                if relatorio_valido and not linha:
                    if len(parametros_usados) >= minimo_parametros:
                        relatorios_filtrados.append("\n".join(relatorio_em_construcao))
                        print(f"Relat√≥rio completo detectado e adicionado.")  # Log para depura√ß√£o
                    else:
                        print("Aviso: Relat√≥rio incompleto ignorado.")
                    relatorio_valido = False

        # Adiciona o √∫ltimo relat√≥rio se atender ao m√≠nimo de par√¢metros
        if relatorio_valido and len(parametros_usados) >= minimo_parametros:
            relatorios_filtrados.append("\n".join(relatorio_em_construcao))
            print(f"√öltimo relat√≥rio completo adicionado.")
        else:
            print("Aviso: √öltimo relat√≥rio ignorado por estar incompleto.")

        # Salva os relat√≥rios filtrados no arquivo de sa√≠da
        with open(arquivo_saida, "w", encoding="utf-8") as saida:
            for relatorio in relatorios_filtrados:
                saida.write(relatorio + "\n\n")
        print(f"Relat√≥rios filtrados com sucesso: {len(relatorios_filtrados)}")

        # Adiciona informa√ß√µes irrelevantes ao arquivo sem sobrescrever
        with open(arquivo_irrelevante, "a", encoding="utf-8") as irrelevantes:
            for linha in linhas_irrelevantes:
                irrelevantes.write(linha + "\n")
        print(f"Linhas irrelevantes adicionadas ao arquivo: {len(linhas_irrelevantes)}")

    except Exception as e:
        print(f"Erro durante o processamento: {e}")

def validar_arquivos(arquivo_filtrado, arquivo_irrelevante, parametros_estrutura, arquivo_validacao):
    """Valida se informa√ß√µes importantes ou irrelevantes foram classificadas incorretamente e salva no log."""
    problemas_detectados = False
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    try:
        with open(arquivo_irrelevante, "r", encoding="utf-8") as irrelevantes:
            linhas_irrelevantes = [linha.strip() for linha in irrelevantes if linha.strip()]
        linhas_importantes_no_irrelevante = [
            linha for linha in linhas_irrelevantes
            if any(linha.lower().startswith(parametro.lower() + ":") for parametro in parametros_estrutura)
        ]

        with open(arquivo_filtrado, "r", encoding="utf-8") as filtrado:
            linhas_filtradas = [linha.strip() for linha in filtrado if linha.strip()]
        linhas_irrelevantes_no_filtrado = [
            linha for linha in linhas_filtradas
            if not any(linha.lower().startswith(parametro.lower() + ":") for parametro in parametros_estrutura)
        ]

        with open(arquivo_validacao, "a", encoding="utf-8") as log:
            log.write(f"\n=== Valida√ß√£o realizada em {timestamp} ===\n")
            if linhas_importantes_no_irrelevante:
                problemas_detectados = True
                log.write("Informa√ß√µes importantes detectadas em 'informacoes_indesejadas.txt':\n")
                for linha in linhas_importantes_no_irrelevante:
                    log.write(f" - {linha}\n")

            if linhas_irrelevantes_no_filtrado:
                problemas_detectados = True
                log.write("Informa√ß√µes irrelevantes detectadas em 'relatorio_filtrado.txt':\n")
                for linha in linhas_irrelevantes_no_filtrado:
                    log.write(f" - {linha}\n")

        if problemas_detectados:
            print(f"Avisos encontrados. Consulte o arquivo de valida√ß√£o: {arquivo_validacao}")
        else:
            print("Valida√ß√£o conclu√≠da com sucesso. Nenhum problema encontrado.")

    except Exception as e:
        print(f"Erro durante a valida√ß√£o dos arquivos: {e}")

# Caminhos Din√¢micos
diretorio_base = Path.cwd()
pasta_parametros = diretorio_base / "arquivos-gerados" / "arquivos-txt" / "parametros"
pasta_listas = diretorio_base / "arquivos-gerados" / "arquivos-txt" / "listas-txt"
pasta_variacoes = diretorio_base / "arquivos-gerados" / "arquivos-txt" / "variacoes"

# Cria√ß√£o das pastas, se necess√°rio
pasta_parametros.mkdir(exist_ok=True, parents=True)
pasta_listas.mkdir(exist_ok=True, parents=True)
pasta_variacoes.mkdir(exist_ok=True, parents=True)

# Caminhos dos arquivos
caminho_estrutura = pasta_parametros / "estrutura-rdo.txt"
arquivo_entrada = pasta_listas / "wpp.txt"
arquivo_saida = pasta_listas / "relatorio_filtrado.txt"
arquivo_irrelevante = pasta_variacoes / "informacoes_indesejadas.txt"
arquivo_validacao = pasta_variacoes / "log_validacao.txt"

# Carrega a estrutura do relat√≥rio
parametros_estrutura = carregar_estrutura_rdo(caminho_estrutura)
if parametros_estrutura:
    filtrar_relatorio_completo(arquivo_entrada, arquivo_saida, parametros_estrutura, arquivo_irrelevante, minimo_parametros=10)
    validar_arquivos(arquivo_saida, arquivo_irrelevante, parametros_estrutura, arquivo_validacao)


Estrutura carregada com 40 par√¢metros.
Processando linha: 19/01/2025 23:22 - As mensagens e as liga√ß√µes s√£o protegidas com a criptografia de ponta a ponta e ficam somente entre voc√™ e os participantes desta conversa. Nem mesmo o WhatsApp pode l√™-las ou ouvi-las. Toque para saber mais.
Processando linha: 13/03/2024 18:25 - Voc√™ criou este grupo
Processando linha: 20/01/2025 10:24 - +55 31 9781-5761: ---------------------------------------------------
Processando linha: RETORNO DAS ATIVIDADES
Processando linha: ---------------------------------------------------
Processando linha: DATA DE EXECU√á√ÉO:20/01/2025
Par√¢metro detectado: DATA DE EXECU√á√ÉO
Processando linha: STATUS DA EXECU√á√ÉO: desmontagem
Par√¢metro detectado: STATUS DA EXECU√á√ÉO
Processando linha: STATUS DO PLANEJAMENTO:Programada
Par√¢metro detectado: STATUS DO PLANEJAMENTO
Processando linha: TAG:iB-02 _PE-03
Par√¢metro detectado: TAG
Processando linha: ORDEM:202500155907
Par√¢metro detectado: ORDEM
Processando li

**REV-2.0.0 - LIB. P\ TESTE: CONVERTER ARQUIVOS RELATORIO_FILTRADO.TXT PARA RELATORIO_TRATADO.csv**

In [5]:
import csv
import re
from pathlib import Path
from collections import defaultdict
from datetime import datetime

def contar_retorno_atividades(arquivo_entrada):
    """Conta quantos relat√≥rios existem com base na palavra-chave 'RETORNO DAS ATIVIDADES'."""
    with arquivo_entrada.open("r", encoding="utf-8") as entrada:
        return sum(1 for linha in entrada if "RETORNO DAS ATIVIDADES" in linha.upper())

def processar_linha(linha, padroes, dados_atual):
    """Processa uma linha e atualiza os dados coletados com base nos padr√µes."""
    encontrou_valor = False
    for coluna, padrao in padroes.items():
        match = padrao.search(linha)
        if match:
            dados_atual[coluna] = match.group(1).strip()
            encontrou_valor = True
    return encontrou_valor

def salvar_dados(writer, dados_atual):
    """Salva os dados coletados no CSV caso haja valores preenchidos."""
    if any(dados_atual.values()):
        writer.writerow(dados_atual)
        print(f"[LOG {datetime.now()}] Salvando relat√≥rio: {dados_atual}")

def verificar_duplicados_e_faltas(arquivo_csv, total_relatorios):
    """Verifica se o n√∫mero de relat√≥rios no CSV corresponde ao n√∫mero esperado."""
    with arquivo_csv.open("r", encoding="utf-8-sig") as csvfile:
        reader = csv.DictReader(csvfile)
        linhas = list(reader)
        if len(linhas) != total_relatorios:
            print(f"[ALERTA {datetime.now()}] Diferen√ßa detectada: CSV possui {len(linhas)} relat√≥rios, esperado: {total_relatorios}")
        return len(linhas) != total_relatorios

def gerar_csv_detalhado(arquivo_entrada, arquivo_saida_csv):
    colunas = [
        "DATA DE EXECU√á√ÉO", "STATUS DA EXECU√á√ÉO", "STATUS DO PLANEJAMENTO", "TAG",
        "ORDEM", "DESCRI√á√ÉO", "QTD PESSOAS", "TIPO DE ANDAIME", "QUANTIDADE DE ANDAIMES",
        "COMPRIMENTO", "LARGURA", "ALTURA",
        "HOR√ÅRIO DE DISPONIBILIDADE", "HOR√ÅRIO DE IN√çCIO DA ATIVIDADE", "HOR√ÅRIO DE FINALIZA√á√ÉO DA ATIVIDADE",
        "PREPARA√á√ÉO DE MATERIAL", "RAZ√ÉO DO CANCELAMENTO", "HOR√ÅRIO DO CANCELAMENTO",
        "RAZ√ÉO DO DESVIO", "HOR√ÅRIO DO DESVIO",
        "HOR√ÅRIO DE SOLICITA√á√ÉO DA PTS", "HOR√ÅRIO DE LIBERA√á√ÉO DA PTS",
        "HOR√ÅRIO DE SOLICITA√á√ÉO DO BLOQUEIO", "HOR√ÅRIO DE LIBERA√á√ÉO DO BLOQUEIO",
        "HOR√ÅRIO DE SOLICITA√á√ÉO DA LIMPEZA", "HOR√ÅRIO DE LIBERA√á√ÉO DA LIMPEZA",
        "HOR√ÅRIO DE SOLICITA√á√ÉO DA OM", "HOR√ÅRIO DE RECEBIMENTO DA OM",
        "HOR√ÅRIO DE SOLICITA√á√ÉO DA PET", "HOR√ÅRIO DE LIBERA√á√ÉO DA PET"
    ]

    padroes = {coluna: re.compile(fr"{coluna}: ?(.*)", re.IGNORECASE) for coluna in colunas}

    try:
        if not arquivo_entrada.exists():
            print(f"[ERRO {datetime.now()}] Arquivo de entrada '{arquivo_entrada}' n√£o encontrado.")
            return

        total_relatorios = contar_retorno_atividades(arquivo_entrada)
        print(f"[LOG {datetime.now()}] Total de relat√≥rios identificados: {total_relatorios}")

        with arquivo_entrada.open("r", encoding="utf-8") as entrada, arquivo_saida_csv.open("w", encoding="utf-8-sig", newline="") as saida_csv:
            writer = csv.DictWriter(saida_csv, fieldnames=colunas)
            writer.writeheader()

            dados_atual = defaultdict(str)
            for linha in entrada:
                linha = linha.strip()

                if not linha or linha.startswith("-----"):  # Ignora separadores ou linhas vazias
                    continue

                if processar_linha(linha, padroes, dados_atual):
                    print(f"[DEBUG {datetime.now()}] Linha processada: {linha}")

                # Detecta in√≠cio de um novo relat√≥rio e salva o atual
                if "RETORNO DAS ATIVIDADES" in linha.upper():
                    salvar_dados(writer, dados_atual)
                    dados_atual = defaultdict(str)  # Reseta para o pr√≥ximo relat√≥rio

            # Salva o √∫ltimo relat√≥rio
            salvar_dados(writer, dados_atual)

        # Verificar duplicados ou diferen√ßas
        if verificar_duplicados_e_faltas(arquivo_saida_csv, total_relatorios):
            print(f"[ERRO {datetime.now()}] O n√∫mero de relat√≥rios no CSV n√£o corresponde ao esperado.")

        print(f"[LOG {datetime.now()}] Arquivo CSV gerado com sucesso em: {arquivo_saida_csv}")

    except FileNotFoundError as e:
        print(f"[ERRO {datetime.now()}] Arquivo ou diret√≥rio n√£o encontrado: {e}")
    except Exception as e:
        print(f"[ERRO {datetime.now()}] Erro durante o processamento: {e}")

# Caminhos Din√¢micos
base_dir = Path.cwd()

diretorio_entrada = base_dir / "arquivos-gerados" / "arquivos-txt" / "listas-txt"
diretorio_saida = base_dir / "arquivos-gerados" / "arquivos-csv"

arquivo_entrada = diretorio_entrada / "relatorio_filtrado.txt"
arquivo_saida_csv = diretorio_saida / "relatorio_tratado.csv"

diretorio_saida.mkdir(parents=True, exist_ok=True)

gerar_csv_detalhado(arquivo_entrada, arquivo_saida_csv)


[LOG 2025-01-27 04:58:16.203750] Total de relat√≥rios identificados: 55
[DEBUG 2025-01-27 04:58:16.205748] Linha processada: DATA DE EXECU√á√ÉO:20/01/2025
[DEBUG 2025-01-27 04:58:16.205748] Linha processada: STATUS DA EXECU√á√ÉO: desmontagem
[DEBUG 2025-01-27 04:58:16.205748] Linha processada: STATUS DO PLANEJAMENTO:Programada
[DEBUG 2025-01-27 04:58:16.205748] Linha processada: TAG:iB-02 _PE-03
[DEBUG 2025-01-27 04:58:16.205748] Linha processada: ORDEM:202500155907
[DEBUG 2025-01-27 04:58:16.205748] Linha processada: DESCRI√á√ÉO:realizar montagem e desmontagem de andaime para atender atividade rod√≠zio peneira
[DEBUG 2025-01-27 04:58:16.205748] Linha processada: QTD PESSOAS:04
[DEBUG 2025-01-27 04:58:16.205748] Linha processada: TIPO DE ANDAIME:
[DEBUG 2025-01-27 04:58:16.205748] Linha processada: QUANTIDADE DE ANDAIMES:01
[DEBUG 2025-01-27 04:58:16.205748] Linha processada: COMPRIMENTO:3.50
[DEBUG 2025-01-27 04:58:16.205748] Linha processada: LARGURA:2.50
[DEBUG 2025-01-27 04:58:16.2

**REV-1.0.1 - LIB. P\ OPERAR: PADRONIZAR MAI√öSCULO NO ARQUIVO RELATORIO_TRATADO.CSV**

In [6]:
import pandas as pd

# Caminho do arquivo original e do novo arquivo de sa√≠da
arquivo_saida = "/content/arquivos-gerados/arquivos-csv/relatorio_tratado.csv"
arquivo_caixa_alta = "/content/arquivos-gerados/arquivos-csv/relatorio_tratado.csv"

# Carregar o arquivo original
df = pd.read_csv(arquivo_saida)

# Converter todas as colunas de texto para caixa alta
df = df.applymap(lambda x: x.upper() if isinstance(x, str) else x)

# Salvar o novo arquivo com os dados em caixa alta
df.to_csv(arquivo_caixa_alta, index=False)

print(f"Arquivo gerado com sucesso em: {arquivo_caixa_alta}")


FileNotFoundError: [Errno 2] No such file or directory: '/content/arquivos-gerados/arquivos-csv/relatorio_tratado.csv'

**REV-1.0.1 - LIB. P\ OPERAR: TRATAR COLUNA DATAS DO ARQUIVO RELATORIO_TRATADO.CSV**

In [60]:
import pandas as pd
from datetime import datetime

# Caminho do arquivo de entrada
arquivo_entrada = "/content/arquivos-gerados/arquivos-csv/relatorio_tratado.csv"
# Caminho do arquivo de sa√≠da (novo arquivo gerado ap√≥s o tratamento)
arquivo_saida = "/content/arquivos-gerados/arquivos-csv/relatorio_tratado.csv"

# Fun√ß√£o para padronizar as datas
def padronizar_data(data):
    try:
        # Remover espa√ßos extras e barras duplicadas
        data = str(data).strip()
        # Remove qualquer espa√ßo antes e depois das barras
        data = data.replace(" //", "/").replace("//", "/").replace("/ ", "/")
        data = data.replace(" ", "")  # Remover qualquer espa√ßo extra em toda a data

        # Verificar se a data cont√©m o ano (ex.: "03/01/2024" ou "03/01/25")
        if len(data.split("/")) == 3:  # Se tem 3 partes (dia/m√™s/ano)
            dia, mes, ano = data.split("/")
            # Corrigir anos com 2 d√≠gitos, como "25" para "2025"
            if len(ano) == 2:
                ano = "20" + ano
            # Padronizar para o formato DD/MM/YYYY
            return datetime.strptime(f"{dia}/{mes}/{ano}", "%d/%m/%Y").strftime("%d/%m/%Y")

        # Corrigir datas com apenas dia e m√™s (como "06/12")
        if len(data.split("/")) == 2:
            data = data + "/" + str(datetime.now().year)  # Adicionar o ano atual
            return datetime.strptime(data, "%d/%m/%Y").strftime("%d/%m/%Y")

        return data
    except Exception as e:
        # Caso o valor n√£o seja uma data v√°lida, retorna o valor original
        return data

# Carregar o arquivo de entrada
df = pd.read_csv(arquivo_entrada)

# Verificar a coluna de datas e padronizar
if 'DATA DE EXECU√á√ÉO' in df.columns:
    for index, data in enumerate(df['DATA DE EXECU√á√ÉO']):
        data_padronizada = padronizar_data(data)
        # Se a data padronizada for diferente da original, vamos atualizar
        if data_padronizada != str(data).strip():
            df.at[index, 'DATA DE EXECU√á√ÉO'] = data_padronizada

# Corrigir datas de 2024 que s√£o inconsistentes
for index, data in enumerate(df['DATA DE EXECU√á√ÉO']):
    try:
        data_formatada = datetime.strptime(data, "%d/%m/%Y")
        # Corrigir datas muito antigas, por exemplo, 03/01/2024 quando deveriam ser de 2025
        if data_formatada.year == 2024 and data_formatada < datetime(2025, 1, 1):
            df.at[index, 'DATA DE EXECU√á√ÉO'] = (data_formatada.replace(year=2025)).strftime("%d/%m/%Y")
    except Exception:
        continue  # Se a data n√£o estiver no formato esperado, ignore a linha

# Salvar o arquivo corrigido
df.to_csv(arquivo_saida, index=False)

print(f"Datas corrigidas e arquivo atualizado com sucesso! Arquivo salvo em: {arquivo_saida}")


Datas corrigidas e arquivo atualizado com sucesso! Arquivo salvo em: /content/arquivos-gerados/arquivos-csv/relatorio_tratado.csv


**REV-1.0.1 - LIB. P\ OPERAR: TRATAR COLUNA STATUS DA EXECU√á√ÉO NO ARQUIVO RELATORIO_TRATADO.CSV**

In [61]:
import pandas as pd

# Fun√ß√£o para tratar os valores de 'status da execu√ß√£o' conforme as regras
def tratar_status(status):
    # Convertendo para min√∫sculas para facilitar a compara√ß√£o
    status = status.lower() if isinstance(status, str) else ""

    # Verificando e atribuindo os novos status conforme as regras fornecidas
    if 'montagem' in status and 'desmontagem' not in status:
        return 'MONTAGEM'  # Montagem sem a presen√ßa de desmontagem
    elif 'desmontagem' in status:
        return 'DESMONTAGEM'  # Desmontagem, independentemente de outras palavras
    elif 'cancelado' in status:
        return 'CANCELADO'
    elif 'adequa√ß√£o' in status:
        return 'ADEQUA√á√ÉO'
    elif '% montado' in status or 'andaime estava 70%' in status or 'andaime em andamento' in status:
        return 'EM ANDAMENTO'
    elif 'finalizado' in status:
        return 'MONTAGEM E DESMONTAGEM'
    elif '70%' in status:
        return 'EM ANDAMENTO'
    elif '95%' in status:
        return 'EM ANDAMENTO'
    elif '100%' in status:
        return 'MONTAGEM E DESMONTAGEM'
    else:
        return status  # Caso n√£o atenda nenhuma regra, mant√©m o valor original

# Caminho do arquivo .csv
caminho_arquivo = "/content/arquivos-gerados/arquivos-csv/relatorio_tratado.csv"

# Carregar o arquivo .csv usando pandas
df = pd.read_csv(caminho_arquivo)

# Limpeza das colunas para garantir que n√£o h√° espa√ßos extras
df.columns = df.columns.str.strip()

# Aplicar a fun√ß√£o de tratamento na coluna 'status da execu√ß√£o'
df['STATUS DA EXECU√á√ÉO'] = df['STATUS DA EXECU√á√ÉO'].apply(tratar_status)

# Exibir as 5 primeiras linhas do DataFrame tratado (opcional)
print("Primeiras linhas do arquivo tratado:")
print(df.head())

# Se precisar salvar o arquivo tratado (opcional)
caminho_saida = "/content/arquivos-gerados/arquivos-csv/relatorio_tratado.csv"
df.to_csv(caminho_saida, index=False)

print(f"Arquivo tratado salvo em: {caminho_saida}")


Primeiras linhas do arquivo tratado:
  DATA DE EXECU√á√ÉO   STATUS DA EXECU√á√ÉO STATUS DO PLANEJAMENTO           TAG  \
0       20/01/2025          DESMONTAGEM             PROGRAMADA  IB-02 _PE-03   
1       20/01/2025          DESMONTAGEM             PROGRAMADA  IB-02 _PE-01   
2       20/01/2025              montado             PROGRAMADA  IB-02 _PE-03   
3       20/01/2025  adequacao concluida                    NaN       FT 6421   
4       20/01/2025              montado                    NaN        FI6811   

          ORDEM                                          DESCRI√á√ÉO  \
0  202500155907  REALIZAR MONTAGEM E DESMONTAGEM DE ANDAIME PAR...   
1  202500155806  REALIZAR MONTAGEM E DESMONTAGEM DE ANDAIME PAR...   
2  202500155907  REALIZAR MONTAGEM E DESMONTAGEM DE ANDAIME PAR...   
3  202500258685                               ADEQUA√á√ÉO NO ANDAIME   
4  202500330903            MONTAGEM DE ANDAIME PARA REFOR√áAR SOLDA   

   QTD PESSOAS TIPO DE ANDAIME  QUANTIDADE DE ANDAIM

**REV-1.0.1 - LIB. P\ OPERAR: TRATAR COLUNA QTD PESSOAS DO ARQUIVO RELATORIO_TRATADO.CSV**

In [62]:
import pandas as pd

# Caminhos dos arquivos no Colab
arquivo_entrada = "/content/arquivos-gerados/arquivos-csv/relatorio_tratado.csv"
arquivo_saida = "/content/arquivos-gerados/arquivos-csv/relatorio_tratado.csv"

# Lista de colunas para verificar
colunas_tratamento = ["QTD PESSOAS", "QUANTIDADE DE ANDAIMES"]

# Fun√ß√£o para padronizar os valores
def padronizar_valores(valor):
    try:
        # Verifica se √© NaN ou vazio
        if pd.isna(valor) or str(valor).strip() == "":
            return "0,00"
        # Verifica se √© texto
        if isinstance(valor, str):
            valor = valor.strip().lower()
            if not valor.replace('.', '').replace(',', '').isdigit():
                return "0,00"
        # Convers√£o para float
        valor = float(valor)
        if valor in [1, 1.0, 1.00]:
            return "1,00"
        elif valor in [3, 3.0, 3.00]:
            return "3,00"
        elif valor in [10, 10.0, 10.00]:
            return "10,00"
        elif valor in [0, 0.0, 0.00]:
            return "0,00"
        # Retorna valores no formato esperado
        return f"{valor:.2f}".replace('.', ',')
    except Exception:
        return "0,00"  # Retorna "0,00" em caso de erro

# Carregar o arquivo de entrada
try:
    df_entrada = pd.read_csv(arquivo_entrada)
except FileNotFoundError:
    print(f"Arquivo de entrada n√£o encontrado em: {arquivo_entrada}")
    raise

# Criar uma c√≥pia do DataFrame para sa√≠da
df_saida = df_entrada.copy()

# Verificar e tratar as colunas
for coluna in colunas_tratamento:
    if coluna in df_entrada.columns:
        # Padronizar valores na coluna
        df_saida[coluna] = df_entrada[coluna].apply(padronizar_valores)
    else:
        print(f"Coluna {coluna} n√£o encontrada no arquivo de entrada.")

# Salvar o arquivo tratado
df_saida.to_csv(arquivo_saida, index=False)

print(f"Arquivo tratado e salvo em: {arquivo_saida}")


Arquivo tratado e salvo em: /content/arquivos-gerados/arquivos-csv/relatorio_tratado.csv


**REV-1.0.1 - LIB. P\ OPERAR: TRATAR HORAS DO ARQUIVO RELATORIO_TRATADO.CSV**

In [63]:
import pandas as pd

# Caminho do arquivo gerado anteriormente
caminho_arquivo = "/content/arquivos-gerados/arquivos-csv/relatorio_tratado.csv"

# Lista de colunas com campos de horas a serem padronizados
colunas_horas = [
    "HOR√ÅRIO DE DISPONIBILIDADE", "HOR√ÅRIO DE IN√çCIO DA ATIVIDADE",
    "HOR√ÅRIO DE FINALIZA√á√ÉO DA ATIVIDADE", "RAZ√ÉO DO CANCELAMENTO",
    "HOR√ÅRIO DO CANCELAMENTO", "HOR√ÅRIO DO DESVIO",
    "HOR√ÅRIO DE SOLICITA√á√ÉO DA PTS", "HOR√ÅRIO DE LIBERA√á√ÉO DA PTS",
    "HOR√ÅRIO DE SOLICITA√á√ÉO DO BLOQUEIO", "HOR√ÅRIO DE LIBERA√á√ÉO DO BLOQUEIO",
    "HOR√ÅRIO DE SOLICITA√á√ÉO DA LIMPEZA", "HOR√ÅRIO DE LIBERA√á√ÉO DA LIMPEZA",
    "HOR√ÅRIO DE SOLICITA√á√ÉO DA OM", "HOR√ÅRIO DE RECEBIMENTO DA OM",
    "HOR√ÅRIO DE SOLICITA√á√ÉO DA PET", "HOR√ÅRIO DE LIBERA√á√ÉO DA PET"
]

# Fun√ß√£o para padronizar os campos de hora
def padronizar_horas(valor):
    try:
        # Verifica se o valor √© string e n√£o est√° vazio
        if isinstance(valor, str) and valor.strip():
            # Se o formato for apenas "HH", completa para "HH:00:00"
            if len(valor) == 2 and valor.isdigit():
                return f"{valor}:00:00"
            # Se o formato for "HH:MM", completa para "HH:MM:00"
            elif len(valor) == 5 and ":" in valor:
                return f"{valor}:00"
            # Verifica se o valor j√° est√° no formato correto "HH:MM:SS"
            elif len(valor) == 8 and ":" in valor:
                return valor
        # Caso n√£o seja v√°lido, retorna o valor padr√£o "00:00:00"
        return "00:00:00"
    except Exception:
        return "00:00:00"

# Carregar o arquivo .csv usando pandas
df = pd.read_csv(caminho_arquivo)

# Garantir que as colunas existem no DataFrame antes de aplicar a fun√ß√£o
for coluna in colunas_horas:
    if coluna in df.columns:
        df[coluna] = df[coluna].apply(padronizar_horas)

# Salvar o arquivo atualizado
caminho_saida = "/content/arquivos-gerados/arquivos-csv/relatorio_tratado.csv"
df.to_csv(caminho_saida, index=False)

print(f"Arquivo tratado com horas padronizadas salvo em: {caminho_saida}")




Arquivo tratado com horas padronizadas salvo em: /content/arquivos-gerados/arquivos-csv/relatorio_tratado.csv


**REV-1.0.1 - LIB. P\ OPERAR: TRATAR DIMENS√ïES DO ARQUIVO RELATORIO_TRATADO.CSV**

In [64]:
import pandas as pd

# Caminhos dos arquivos no Colab
caminho_arquivo = "/content/arquivos-gerados/arquivos-csv/relatorio_tratado.csv"
caminho_saida = "/content/arquivos-gerados/arquivos-csv/relatorio_tratado.csv"

# Lista de colunas de medidas para padronizar
colunas_medidas = ["COMPRIMENTO", "LARGURA", "ALTURA"]

# Fun√ß√£o para padronizar medidas
def padronizar_medidas(valor):
    try:
        # Verifica se o valor est√° vazio ou n√£o num√©rico
        if pd.isna(valor) or str(valor).strip() == "":
            return "0,00"  # Preenche com 0,00
        # Converte para float e depois para o formato desejado
        valor_float = float(str(valor).replace(",", "."))
        return f"{valor_float:,.2f}".replace(".", ",")  # Padroniza para formato 0,00
    except Exception:
        return "0,00"  # Caso erro, retorna 0,00

# Carregar o arquivo original
df = pd.read_csv(caminho_arquivo)

# Padronizar as colunas de medidas
for coluna in colunas_medidas:
    if coluna in df.columns:
        df[coluna] = df[coluna].apply(padronizar_medidas)

# Salvar o arquivo com as medidas padronizadas
df.to_csv(caminho_saida, index=False)

print(f"Arquivo com medidas padronizadas salvo em: {caminho_saida}")



Arquivo com medidas padronizadas salvo em: /content/arquivos-gerados/arquivos-csv/relatorio_tratado.csv


**REV-1.0.1 - LIB. P\ OPERAR: TRANSOFRMAR O ARQUIVO RELATORIO_TRATADO.CSV EM RELATORIO_TRATADO.XLSX**

- INFORMA√á√ïES ADICIONAIS PARA TRATAMENTO: A PLANILHA TEM QUE SER TRATADA NO POWER QUERY PARA IDENTIFICAR DADOS COMO DATA/VALORES/HORAS.

In [65]:
import pandas as pd

# Caminho do arquivo CSV e do novo arquivo Excel
arquivo_csv = "/content/arquivos-gerados/arquivos-csv/relatorio_tratado.csv"
arquivo_excel = "/content/arquivos-gerados/arquivos-xls/relatorio_tratado.xlsx"

# Ler o arquivo CSV
df = pd.read_csv(arquivo_csv)

# Converter para Excel
df.to_excel(arquivo_excel, index=False, engine='openpyxl')

print(f"Arquivo Excel criado com sucesso em: {arquivo_excel}")


Arquivo Excel criado com sucesso em: /content/arquivos-gerados/arquivos-xls/relatorio_tratado.xlsx


**REV-1.0.1 - LIB. P\ OPERAR: CRIAR COLUNA STATUS DA EXECU√á√ÉO NO ARQUIVO RELATORIO_TRATADO.XLSX**

- INFORMA√á√ïES ADICIONAIS SOBRE O PROCESSO: COM BASE NA MONTAGEM, DESMONTAGEM, MONTAGEM E DESMONTAGEM, ADEQUA√á√ÉO E CANCELAMENTO O C√ìDIGO CRIARA UMA COLUNA DETERMINANDO SE A TAREFA FOI EXECUDA OU CANCELADA.
- AINDA SE FAZ NECESS√ÅRIO O TRATAMENTO DE ALGUNSS DADOS MANUALMENTE.

In [66]:
import pandas as pd

# Fun√ß√£o para criar a coluna 'RETORNO DA EXECU√á√ÉO'
def criar_coluna_retorno(arquivo_excel):
    # Carregar o arquivo Excel
    try:
        df = pd.read_excel(arquivo_excel)
        print("Arquivo carregado com sucesso.")
    except Exception as e:
        print(f"Erro ao ler o arquivo: {e}")
        return

    # Exibir as primeiras linhas para depura√ß√£o
    print("Primeiras linhas do arquivo:")
    print(df.head())

    # Verificar se a coluna 'STATUS DA EXECU√á√ÉO' existe, removendo espa√ßos extras
    if 'STATUS DA EXECU√á√ÉO' not in df.columns:
        print("A coluna 'STATUS DA EXECU√á√ÉO' n√£o foi encontrada no arquivo. Verificando se h√° espa√ßos extras...")
        df.columns = df.columns.str.strip()  # Remover espa√ßos extras nos nomes das colunas
        if 'STATUS DA EXECU√á√ÉO' not in df.columns:
            print("A coluna 'STATUS DA EXECU√á√ÉO' ainda n√£o foi encontrada ap√≥s remo√ß√£o de espa√ßos.")
            return
        else:
            print("Espa√ßos extras removidos. Coluna 'STATUS DA EXECU√á√ÉO' encontrada.")

    # Definir as palavras relacionadas a "CANCELADO"
    cancelado_palavras = ["CANCELADO", "CANCELADA", "CANCELL", "CANCEL", "CANCELO"]

    # Definir as palavras relacionadas a "EXECUTADO"
    executado_palavras = ["MONTADO", "DESMONTADO", "ADEQUA√á√ÉO", "EXECUTADO", "MONTAGEM", "DESMONTA"]

    # Criar a nova coluna 'RETORNO DA EXECU√á√ÉO' com valores default vazios
    df['RETORNO DA EXECU√á√ÉO'] = ""

    # Verificar o campo 'STATUS DA EXECU√á√ÉO' e atribuir valores √† nova coluna
    for index, row in df.iterrows():
        status_execucao = str(row['STATUS DA EXECU√á√ÉO']).upper()  # Converter para mai√∫sculas para compara√ß√£o

        # Verificar se o status est√° relacionado a "CANCELADO"
        if any(palavra in status_execucao for palavra in cancelado_palavras):
            df.at[index, 'RETORNO DA EXECU√á√ÉO'] = "CANCELADO"

        # Verificar se o status est√° relacionado a "EXECUTADO"
        elif any(palavra in status_execucao for palavra in executado_palavras):
            df.at[index, 'RETORNO DA EXECU√á√ÉO'] = "EXECUTADO"

    # Exibir as primeiras linhas ap√≥s adicionar a coluna
    print("Primeiras linhas ap√≥s adicionar a coluna 'RETORNO DA EXECU√á√ÉO':")
    print(df.head())

    # Salvar o arquivo modificado no mesmo local
    try:
        df.to_excel(arquivo_excel, index=False, engine='openpyxl')
        print(f"Coluna 'RETORNO DA EXECU√á√ÉO' adicionada com sucesso no arquivo: {arquivo_excel}")
    except Exception as e:
        print(f"Erro ao salvar o arquivo: {e}")

# Caminho do arquivo no Colab
arquivo_excel = "/content/arquivos-gerados/arquivos-xls/relatorio_tratado.xlsx"

# Chamar a fun√ß√£o para processar o arquivo
criar_coluna_retorno(arquivo_excel)


Arquivo carregado com sucesso.
Primeiras linhas do arquivo:
  DATA DE EXECU√á√ÉO   STATUS DA EXECU√á√ÉO STATUS DO PLANEJAMENTO           TAG  \
0       20/01/2025          DESMONTAGEM             PROGRAMADA  IB-02 _PE-03   
1       20/01/2025          DESMONTAGEM             PROGRAMADA  IB-02 _PE-01   
2       20/01/2025              montado             PROGRAMADA  IB-02 _PE-03   
3       20/01/2025  adequacao concluida                    NaN       FT 6421   
4       20/01/2025              montado                    NaN        FI6811   

          ORDEM                                          DESCRI√á√ÉO  \
0  202500155907  REALIZAR MONTAGEM E DESMONTAGEM DE ANDAIME PAR...   
1  202500155806  REALIZAR MONTAGEM E DESMONTAGEM DE ANDAIME PAR...   
2  202500155907  REALIZAR MONTAGEM E DESMONTAGEM DE ANDAIME PAR...   
3  202500258685                               ADEQUA√á√ÉO NO ANDAIME   
4  202500330903            MONTAGEM DE ANDAIME PARA REFOR√áAR SOLDA   

  QTD PESSOAS TIPO DE ANDAIME

**REV-1.0.1 - LIB. P\ OPERAR: ORGANIZAR COLUNA STATUS DA EXECU√á√ÉO NO ARQUIVO RELATORIO_TRATADO.XLSX**

- FUNCIONAMENTO DA ETAPA: MOVE A COLUNA STATUS DA EXECU√á√ÉO DA COLUNA 31 PARA A COLUNA 4, TORNANDO F√ÅCIL IDENTIFICAR OS DADOS.

In [67]:
import pandas as pd

# Fun√ß√£o para reorganizar as colunas conforme nova l√≥gica
def reorganizar_colunas_nova_logica(arquivo_excel):
    # Carregar o arquivo Excel
    try:
        df = pd.read_excel(arquivo_excel)
        print("Arquivo carregado com sucesso.")
    except Exception as e:
        print(f"Erro ao ler o arquivo: {e}")
        return

    # Exibir as primeiras linhas para depura√ß√£o
    print("Primeiras linhas do arquivo:")
    print(df.head())

    # Verificar se existem pelo menos 31 colunas
    if len(df.columns) < 31:
        print("Erro: O arquivo precisa ter pelo menos 31 colunas.")
        return

    # Identificar a coluna de √≠ndice 31 (considerando a indexa√ß√£o 0)
    coluna_31 = df.columns[30]  # 31¬™ coluna (index 30)

    # Mover a coluna 31 para a 4¬™ posi√ß√£o (index 3)
    df.insert(3, coluna_31, df.pop(coluna_31))  # Move para a 4¬™ posi√ß√£o

    # Exibir as primeiras linhas ap√≥s reorganizar
    print("Primeiras linhas ap√≥s reorganizar as colunas:")
    print(df.head())

    # Salvar o arquivo modificado
    try:
        df.to_excel(arquivo_excel, index=False, engine='openpyxl')
        print(f"Colunas reorganizadas com sucesso em: {arquivo_excel}")
    except Exception as e:
        print(f"Erro ao salvar o arquivo: {e}")

# Caminho do arquivo
arquivo_excel = "/content/arquivos-gerados/arquivos-xls/relatorio_tratado.xlsx"

# Chamada para reorganizar as colunas conforme a nova l√≥gica
reorganizar_colunas_nova_logica(arquivo_excel)


Arquivo carregado com sucesso.
Primeiras linhas do arquivo:
  DATA DE EXECU√á√ÉO   STATUS DA EXECU√á√ÉO STATUS DO PLANEJAMENTO           TAG  \
0       20/01/2025          DESMONTAGEM             PROGRAMADA  IB-02 _PE-03   
1       20/01/2025          DESMONTAGEM             PROGRAMADA  IB-02 _PE-01   
2       20/01/2025              montado             PROGRAMADA  IB-02 _PE-03   
3       20/01/2025  adequacao concluida                    NaN       FT 6421   
4       20/01/2025              montado                    NaN        FI6811   

          ORDEM                                          DESCRI√á√ÉO  \
0  202500155907  REALIZAR MONTAGEM E DESMONTAGEM DE ANDAIME PAR...   
1  202500155806  REALIZAR MONTAGEM E DESMONTAGEM DE ANDAIME PAR...   
2  202500155907  REALIZAR MONTAGEM E DESMONTAGEM DE ANDAIME PAR...   
3  202500258685                               ADEQUA√á√ÉO NO ANDAIME   
4  202500330903            MONTAGEM DE ANDAIME PARA REFOR√áAR SOLDA   

  QTD PESSOAS TIPO DE ANDAIME